<a href="https://colab.research.google.com/github/aaubs/ds-master/blob/main/notebooks/M5-db-sql-intro.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import sqlite3
import pandas as pd

# Introduction to the Structured Query Language (SQL)

## What is SQL?

* SQL stands for Structured Query Language
* Used to communicate with relational databases
* Allows you to create, read, update, and delete data
* Commonly used in data analytics, data science, and data engineering


### Relational Databases

* Organize data into tables with rows and columns
* Each row represents a record or an entity
* Each column represents an attribute of the record

Example: 

| id | first_name | last_name | age |
|----|------------|-----------|-----|
| 1  | John       | Doe       | 30  |
| 2  | Jane       | Smith     | 25  |

---

### SQL Syntax Overview

* SELECT: Read data from the database
* INSERT: Add new records to the database
* UPDATE: Modify existing records in the database
* DELETE: Remove records from the database
* CREATE, ALTER, DROP: Manage database structure


# First steps in SQL(ite)

SQLite is a lightweight, serverless, and self-contained database engine that's easy to set up and use within Python applications. In comparison to other SQL dastabases, it does not require a sepperated server.

The sqlite3 module is included in the Python standard library, so you don't need to install any additional packages to use it.

## Create and populate a database

In this example, we'll create a SQLite database for a music streaming service like Spotify. The database will have the following tables: 

1. artists
2. albums
3. tracks
4. playlists
5. playlist_tracks

We'll also insert sample data into these tables. In the schema, we will also define the relationship within and between the tables (primary & foreign key).

Let's start by connecting to the SQLite database and creating a few records in the tables:

In [16]:
# Connect to the SQLite database (creates a new file if it doesn't exist)
conn = sqlite3.connect('music.db')

# Set up a cursor (pointer to rows in database)
c = conn.cursor()

In [17]:
# Create the `artists` table
c.execute('''
CREATE TABLE artists (
    id INTEGER PRIMARY KEY,
    name TEXT
)
''')

# Note: Primary key is the unique identifier of the table

In [18]:
# Insert sample data into the `artists` table
artists_data = [
    (1, 'The Beatles'),
    (2, 'Pink Floyd'),
    (3, 'Led Zeppelin'),
    (4, 'Queen')
]

In [19]:
# with executemany get several rows in at once
c.executemany("INSERT INTO artists (id, name) VALUES (?, ?)", artists_data)

In [20]:
# Create the `albums` table
c.execute('''
CREATE TABLE albums (
    id INTEGER PRIMARY KEY,
    name TEXT,
    artist_id INTEGER,
    release_year INTEGER,
    FOREIGN KEY (artist_id) REFERENCES artists (id)
)
''')

# Note: Foreign key connects to other tables

In [21]:
# Insert sample data into the `albums` table
albums_data = [
    (1, 'Abbey Road', 1, 1969),
    (2, 'The Wall', 2, 1979),
    (3, 'IV', 3, 1971),
    (4, 'A Night at the Opera', 4, 1975)
]

In [22]:
c.executemany("INSERT INTO albums (id, name, artist_id, release_year) VALUES (?, ?, ?, ?)", albums_data)

In [23]:
# Create the `tracks` table
c.execute('''
CREATE TABLE tracks (
    id INTEGER PRIMARY KEY,
    name TEXT,
    album_id INTEGER,
    duration INTEGER,  -- Duration in seconds
    FOREIGN KEY (album_id) REFERENCES albums (id)
)
''')

In [24]:
# Insert sample data into the `tracks` table
tracks_data = [
    (1, 'Come Together', 1, 260),
    (2, 'Something', 1, 182),
    (3, 'Another Brick in the Wall', 2, 231),
    (4, 'Mother', 2, 334),
    (5, 'Stairway to Heaven', 3, 482),
    (6, 'Black Dog', 3, 296),
    (7, 'Bohemian Rhapsody', 4, 355),
    (8, 'You\'re My Best Friend', 4, 163)
]

In [25]:
c.executemany("INSERT INTO tracks (id, name, album_id, duration) VALUES (?, ?, ?, ?)", tracks_data)

In [26]:
# Create the `playlists` table
c.execute('''
CREATE TABLE playlists (
    id INTEGER PRIMARY KEY,
    name TEXT
)
''')

In [27]:
# Insert sample data into the `playlists` table
playlists_data = [
    (1, 'Classic Rock'),
    (2, 'Best of 70s')
]

In [28]:
c.executemany("INSERT INTO playlists (id, name) VALUES (?, ?)", playlists_data)

In [29]:
# Create the `playlist_tracks` table
c.execute('''
CREATE TABLE playlist_tracks (
    id INTEGER PRIMARY KEY,
    playlist_id INTEGER,
    track_id INTEGER,
    FOREIGN KEY (playlist_id) REFERENCES playlists (id),
    FOREIGN KEY (track_id) REFERENCES tracks (id)
)
''')

In [32]:
# Insert sample data into the `playlist_tracks` table
playlist_tracks_data = [
    (1, 1, 1),
    (2, 1, 3),
    (3, 1, 2)
]

In [33]:
c.executemany("INSERT INTO playlist_tracks (id, playlist_id, track_id) VALUES (?, ?,?)", playlist_tracks_data)

In [34]:
# Query the schema for each table from the `sqlite_master` table
c.execute("SELECT name, sql FROM sqlite_master WHERE type='table' ORDER BY name")
schemas = c.fetchall()

# Print the schema for each table
for schema in schemas:
    table_name, table_schema = schema
    print(f"Table: {table_name}\nSchema:\n{table_schema}\n")

Table: albums
Schema:
CREATE TABLE albums (
    id INTEGER PRIMARY KEY,
    name TEXT,
    artist_id INTEGER,
    release_year INTEGER,
    FOREIGN KEY (artist_id) REFERENCES artists (id)
)

Table: artists
Schema:
CREATE TABLE artists (
    id INTEGER PRIMARY KEY,
    name TEXT
)

Table: playlist_tracks
Schema:
CREATE TABLE playlist_tracks (
    id INTEGER PRIMARY KEY,
    playlist_id INTEGER,
    track_id INTEGER,
    FOREIGN KEY (playlist_id) REFERENCES playlists (id),
    FOREIGN KEY (track_id) REFERENCES tracks (id)
)

Table: playlists
Schema:
CREATE TABLE playlists (
    id INTEGER PRIMARY KEY,
    name TEXT
)

Table: tracks
Schema:
CREATE TABLE tracks (
    id INTEGER PRIMARY KEY,
    name TEXT,
    album_id INTEGER,
    duration INTEGER,  -- Duration in seconds
    FOREIGN KEY (album_id) REFERENCES albums (id)
)



## Querying SQL

Now, let's demonstrate some common SQLite commands:

* **SELECT:** Fetch data from a table.

In [37]:
c.execute("SELECT * FROM artists")
artists = c.fetchall()
for artist in artists:
    print(artist)

(1, 'The Beatles')
(2, 'Pink Floyd')
(3, 'Led Zeppelin')
(4, 'Queen')


* **WHERE:** Filter rows based on a condition.


In [38]:
c.execute("SELECT * FROM albums WHERE release_year >= 1970")
albums = c.fetchall()
for album in albums:
    print(album)

(2, 'The Wall', 2, 1979)
(3, 'IV', 3, 1971)
(4, 'A Night at the Opera', 4, 1975)


* **ORDER BY:** Arrange rows based on a column.

In [39]:
c.execute("SELECT * FROM albums ORDER BY release_year DESC")
sorted_albums = c.fetchall()
for album in sorted_albums:
    print(album)

(2, 'The Wall', 2, 1979)
(4, 'A Night at the Opera', 4, 1975)
(3, 'IV', 3, 1971)
(1, 'Abbey Road', 1, 1969)


* **JOIN:** Combine tables based on common columns. (you know that already)

In [41]:
c.execute("""
    SELECT tracks.name AS track_name, albums.name AS album_name, artists.name AS artist_name
    FROM tracks
    JOIN albums ON tracks.album_id = albums.id
    JOIN artists ON albums.artist_id = artists.id
""")
track_details = c.fetchall()
for track in track_details:
    print(track)


('Come Together', 'Abbey Road', 'The Beatles')
('Something', 'Abbey Road', 'The Beatles')
('Another Brick in the Wall', 'The Wall', 'Pink Floyd')
('Mother', 'The Wall', 'Pink Floyd')
('Stairway to Heaven', 'IV', 'Led Zeppelin')
('Black Dog', 'IV', 'Led Zeppelin')
('Bohemian Rhapsody', 'A Night at the Opera', 'Queen')
("You're My Best Friend", 'A Night at the Opera', 'Queen')


* **GROUP BY:** operations on group level (like in pandas).

In [42]:
c.execute("""
    SELECT artists.name, SUM(tracks.duration) AS total_duration
    FROM tracks
    JOIN albums ON tracks.album_id = albums.id
    JOIN artists ON albums.artist_id = artists.id
    GROUP BY artists.id
""")
artist_durations = c.fetchall()
for artist_duration in artist_durations:
    print(artist_duration)

('The Beatles', 442)
('Pink Floyd', 565)
('Led Zeppelin', 778)
('Queen', 518)


Now, let's combine multiple commands into a more complex query. This query will fetch the track names and their artist names, along with the total duration of tracks per artist, for tracks that are part of the "Classic Rock" playlist, ordered by the artist's name:

In [ ]:
sql_query = """
    SELECT
        artists.name AS artist_name,
        tracks.name AS track_name,
        SUM(tracks.duration) OVER (PARTITION BY artists.id) AS total_duration
    FROM tracks
    JOIN albums ON tracks.album_id = albums.id
    JOIN artists ON albums.artist_id = artists.id
    JOIN playlist_tracks ON tracks.id = playlist_tracks.track_id
    JOIN playlists ON playlist_tracks.playlist_id = playlists.id
    WHERE playlists.name = "Classic Rock"
    ORDER BY artists.name
"""

In [43]:
c.execute(sql_query)
results = c.fetchall()
for result in results:
    print(result)

('Pink Floyd', 'Another Brick in the Wall', 231)
('The Beatles', 'Come Together', 442)
('The Beatles', 'Something', 442)


In [ ]:
df = pd.read_sql_query(sql_query, conn)

In [ ]:
# Close comnnection once you are done
conn.close()

## Practice a bit

In [50]:
### useful: download and extract chinook sample DB
import urllib.request
import zipfile
from functools import partial
import os

chinook_url = 'http://www.sqlitetutorial.net/wp-content/uploads/2018/03/chinook.zip'
if not os.path.exists('chinook.zip'):
    print('downloading chinook.zip ', end='')
    with urllib.request.urlopen(chinook_url) as response:
        with open('chinook.zip', 'wb') as f:
            for data in iter(partial(response.read, 4*1024), b''):
                print('.', end='', flush=True)
                f.write(data)

zipfile.ZipFile('chinook.zip').extractall()
assert os.path.exists('chinook.db')

downloading chinook.zip ...........................................................................

in this exercise we're going to experiment with the [Chinook sample DB](http://www.sqlitetutorial.net/sqlite-sample-database/). 

![Chinook](http://www.sqlitetutorial.net/wp-content/uploads/2015/11/sqlite-sample-database-color.jpg)

There are 11 tables in the chinook sample database.

* employees table stores employees data such as employee id, last name, first name, etc. It also has a field named ReportsTo to specify who reports to whom.
* customers table stores customers data.
* invoices & invoice_items tables: these two tables store invoice data. The invoices table stores invoice header data and the invoice_items table stores the invoice line items data.
* artists table stores artists data. It is a simple table that contains only the artist id and name.
* albums table stores data about a list of tracks. Each album belongs to one artist. However, one artist may have multiple albums.
* media_types table stores media types such as MPEG audio and AAC audio files.
* genres table stores music types such as rock, jazz, metal, etc.
* tracks table stores the data of songs. Each track belongs to one album.
* playlists & playlist_track tables: playlists table store data about playlists. Each playlist contains a list of tracks. Each track may belong to multiple playlists. The relationship between the playlists table and tracks table is many-to-many. The playlist_track table is used to reflect this relationship.



In [51]:
# Connect to the SQLite database (creates a new file if it doesn't exist)
conn = sqlite3.connect('chinook.db')

# Set up a cursor (pointer to rows in database)
c = conn.cursor()

In [52]:
# Query the schema for each table from the `sqlite_master` table
c.execute("SELECT name, sql FROM sqlite_master WHERE type='table' ORDER BY name")
schemas = c.fetchall()

# Print the schema for each table
for schema in schemas:
    table_name, table_schema = schema
    print(f"Table: {table_name}\nSchema:\n{table_schema}\n")

Table: albums
Schema:
CREATE TABLE "albums"
(
    [AlbumId] INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL,
    [Title] NVARCHAR(160)  NOT NULL,
    [ArtistId] INTEGER  NOT NULL,
    FOREIGN KEY ([ArtistId]) REFERENCES "artists" ([ArtistId]) 
		ON DELETE NO ACTION ON UPDATE NO ACTION
)

Table: artists
Schema:
CREATE TABLE "artists"
(
    [ArtistId] INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL,
    [Name] NVARCHAR(120)
)

Table: customers
Schema:
CREATE TABLE "customers"
(
    [CustomerId] INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL,
    [FirstName] NVARCHAR(40)  NOT NULL,
    [LastName] NVARCHAR(20)  NOT NULL,
    [Company] NVARCHAR(80),
    [Address] NVARCHAR(70),
    [City] NVARCHAR(40),
    [State] NVARCHAR(40),
    [Country] NVARCHAR(40),
    [PostalCode] NVARCHAR(10),
    [Phone] NVARCHAR(24),
    [Fax] NVARCHAR(24),
    [Email] NVARCHAR(60)  NOT NULL,
    [SupportRepId] INTEGER,
    FOREIGN KEY ([SupportRepId]) REFERENCES "employees" ([EmployeeId]) 
		ON DELETE NO ACTION ON UPDATE NO AC

**Exercise 1:** Find top 10 sold tracks.

In [57]:
sql_query = """
  SELECT tracks."Name", sum(invoice_items."Quantity") AS sold 
  FROM tracks JOIN invoice_items ON tracks."TrackId" = invoice_items."TrackId" 
  GROUP BY tracks."TrackId" 
  ORDER BY sold DESC
  LIMIT 10
"""

df = pd.read_sql_query(sql_query, conn)
df

,Name,sold
0,Balls to the Wall,2
1,Inject The Venom,2
2,Snowballed,2
3,Overdose,2
4,Deuces Are Wild,2
5,Not The Doctor,2
6,Por Causa De Você,2
7,Welcome Home (Sanitarium),2
8,Snowblind,2
9,Cornucopia,2


**Exercise 2:* Top 10 best selling artists by name.

In [58]:
sql_query = """
  SELECT sum(invoice_items."Quantity") AS sold, artists."Name" AS "Artist"  
  FROM invoice_items 
    JOIN tracks ON tracks."TrackId" = invoice_items."TrackId" 
    JOIN albums ON albums."AlbumId" = tracks."AlbumId" 
    JOIN artists ON artists."ArtistId" = albums."ArtistId"
  GROUP BY artists."ArtistId" 
  ORDER BY sold DESC
  LIMIT 10
"""

df = pd.read_sql_query(sql_query, conn)
df

,sold,Artist
0,140,Iron Maiden
1,107,U2
2,91,Metallica
3,87,Led Zeppelin
4,45,Os Paralamas Do Sucesso
5,44,Deep Purple
6,42,Faith No More
7,41,Lost
8,40,Eric Clapton
9,39,R.E.M.


In [59]:
# Close comnnection once you are done
conn.close()

# SQL and ML 

## Traditional ML

In [ ]:
import sqlite3
from sklearn import datasets



In [ ]:
iris = datasets.load_iris()

# Connect to the SQLite database
conn = sqlite3.connect("iris_data.db")
cursor = conn.cursor()

In [68]:
# Create a table for the Iris dataset
cursor.execute("""
CREATE TABLE IF NOT EXISTS iris_data (
    id INTEGER PRIMARY KEY,
    sepal_length REAL,
    sepal_width REAL,
    petal_length REAL,
    petal_width REAL,
    species INTEGER
)
""")

# Insert the Iris dataset into the table
data_to_insert = [tuple(row) + (label,) for row, label in zip(iris.data, iris.target)]
cursor.executemany("""
INSERT INTO iris_data (sepal_length, sepal_width, petal_length, petal_width, species) VALUES (?, ?, ?, ?, ?)
""", data_to_insert)

conn.commit()

In [63]:
import numpy as np
from sklearn.model_selection import train_test_split

cursor.execute("SELECT sepal_length, sepal_width, petal_length, petal_width, species FROM iris_data")
data = cursor.fetchall()

X, y = zip(*[(row[:4], row[4]) for row in data])
X = np.array(X)
y = np.array(y)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1337)

In [64]:
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

model = SVC()
model.fit(X_train, y_train)

y_pred = model.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.2f}")

Accuracy: 0.97


## Deep Learning: Reading in batches from DB

Ok, so far so good. Would it not be cool if we could sequentially feed in data to train and/or predict from the database. Then we coulkd work with really big datasets without running in RAM issues.

We will now train a simple neural network model using PyTorch. We'll create a custom dataset class that fetches data from the SQLite database and loads it in batches using a DataLoader.

In [74]:
import torch
from torch.utils.data import Dataset

class IrisDataset(Dataset):
    def __init__(self, cursor, indices):
        self.cursor = cursor
        self.indices = indices

    def __len__(self):
        return len(self.indices)

    def __getitem__(self, idx):
        index = self.indices[idx]
        self.cursor.execute("SELECT sepal_length, sepal_width, petal_length, petal_width, species FROM iris_data WHERE id=?", (index + 1,))
        row = self.cursor.fetchone()
        x = torch.tensor(row[:4], dtype=torch.float32)
        y = torch.tensor(row[4], dtype=torch.long)
        return x, y

In [75]:
from torch.utils.data import DataLoader, SubsetRandomSampler

# Total number of samples in the dataset
n_samples = len(data)
indices = list(range(n_samples))

# Split indices into training and testing sets
train_indices, test_indices = train_test_split(indices, test_size=0.2, random_state=42)

# Create DataLoaders
train_sampler = SubsetRandomSampler(train_indices)
test_sampler = SubsetRandomSampler(test_indices)

train_dataset = IrisDataset(cursor, train_indices)
test_dataset = IrisDataset(cursor, test_indices)

train_loader = DataLoader(train_dataset, batch_size=10, sampler=train_sampler)
test_loader = DataLoader(test_dataset, batch_size=10, sampler=test_sampler)

NameError: ignored